## Step 1: Read the csv file and convert it to a feature class in GIS

In [10]:
import csv

file = open("boundary.csv")
csv_reader = csv.reader(file)
for line in csv_reader:
    print(line)

['col', 'row', 'X', 'Y']
['4871', '174', '699102.8877924071', '186780.44581266836']
['4871', '174', '699102.8877924071', '186780.44581266836']
['4872', '174', '699105.8874190656', '186780.44581266836']
['4870', '175', '699099.8881657487', '186777.44618600988']
['4873', '174', '699108.8870457241', '186780.44581266836']
['4869', '175', '699096.8885390902', '186777.44618600988']
['4874', '174', '699111.8866723826', '186780.44581266836']
['4868', '175', '699093.8889124317', '186777.44618600988']
['4868', '175', '699093.8889124317', '186777.44618600988']
['4875', '174', '699114.886299041', '186780.44581266836']
['4876', '174', '699117.8859256995', '186780.44581266836']
['4876', '174', '699117.8859256995', '186780.44581266836']
['4867', '176', '699090.8892857733', '186774.44655935143']
['4876', '175', '699117.8859256995', '186777.44618600988']
['4866', '176', '699087.8896591148', '186774.44655935143']
['4865', '176', '699084.8900324563', '186774.44655935143']
['4877', '176', '699120.88555235

In [2]:
import arcpy
arcpy.da.Describe("flood_2class.tif")

{'catalogPath': 'flood_2class.tif',
 'FIDSet': None,
 'size': 1089313,
 'dateCreated': '2025-05-11T19:34:51.000',
 'dateAccessed': '2025-05-11T20:46:01.000',
 'dateModified': '2024-12-18T03:57:40.000',
 'workspace': <geoprocessing describe data object at 0x296efd76bf0>,
 'supportsBigObjectID': False,
 'supportsBigInteger': False,
 'supportsTimeOnly': False,
 'supportsDateOnly': False,
 'supportsTimestampOffset': False,
 'bandCount': 1,
 'baseName': 'flood_2class',
 'canVersion': False,
 'changeTracked': False,
 'children': [{'catalogPath': 'flood_2class.tif\\Band_1',
   'FIDSet': None,
   'workspace': <geoprocessing describe data object at 0x296f97a8af0>,
   'supportsBigObjectID': False,
   'supportsBigInteger': False,
   'supportsTimeOnly': False,
   'supportsDateOnly': False,
   'supportsTimestampOffset': False,
   'baseName': 'Band_1',
   'canVersion': False,
   'changeTracked': False,
   'children': [],
   'childrenExpanded': True,
   'dataElementType': 'DERasterBand',
   'datasetT

In [9]:
desc = arcpy.Describe("flood_2class.tif")
sr = desc.spatialReference
sr

name (Projected Coordinate System),NAD_1983_StatePlane_North_Carolina_FIPS_3200
factoryCode (WKID),32119
linearUnitName (Linear Unit),Meter
name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [11]:
arcpy.env.workspace = r"F:\OneDrive - Louisiana State University\LSU_Course\GEOG\GEOG 4057 GIS Programming\GEOG4057_Hanqi\Projects\Project_2_Hanqi"
import os
input = os.path.join(arcpy.env.workspace, "boundary.csv")
# input = "boundary.csv"
output = os.path.join(arcpy.env.workspace, "boundary_pnts.shp")
arcpy.management.XYTableToPoint(input, output, "X", "Y", coordinate_system=sr)
# arcpy.management.Delete("boundary_pnts.shp")

<Result 'F:\\OneDrive - Louisiana State University\\LSU_Course\\GEOG\\GEOG 4057 GIS Programming\\GEOG4057_Hanqi\\Projects\\Project_2_Hanqi\\boundary_pnts.shp'>

Alternative solution   

In [12]:
# !pip install pandas geopandas shapely

  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

csv_file = "boundary.csv"
data = pd.read_csv(csv_file)

if not{"X","Y"}.issubset(data.columns):
    raise ValueError("X and Y columns are required in the CSV file.")

# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(data["X"], data["Y"])]

gdf = gpd.GeoDataFrame(data, geometry=geometry)
gdf.crs = "EPSG:32119"  # Set the coordinate reference system (CRS) to WGS84

  
gdf.to_file("boundary_pnts_geopandas.shp", driver="ESRI Shapefile")

print(gdf.head())


    col  row              X              Y                       geometry
0  4871  174  699102.887792  186780.445813  POINT (699102.888 186780.446)
1  4871  174  699102.887792  186780.445813  POINT (699102.888 186780.446)
2  4872  174  699105.887419  186780.445813  POINT (699105.887 186780.446)
3  4870  175  699099.888166  186777.446186  POINT (699099.888 186777.446)
4  4873  174  699108.887046  186780.445813  POINT (699108.887 186780.446)


## Step 2: Retriebe data from ee

In [94]:
import ee
ee.Authenticate()

True

In [18]:
ee. Initialize(project='ee-hanqi')

https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m   
Find the Earth Engine Snippet: ee.Image("USGS/3DEP/10m")

In [20]:
dem = ee.Image("USGS/3DEP/10m")

In [22]:
dem.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [3650412, 939612],
   'crs': 'EPSG:4269',
   'crs_transform': [9.259259259299957e-05,
    0,
    -174.0005555570324,
    0,
    -9.259259259299957e-05,
    72.00055555584566]}],
 'version': 1648043371709135,
 'id': 'USGS/3DEP/10m',
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[8.533243855035293, 72.00060200570647],
    [-36.52255593943564, 72.00060196804901],
    [-106.00421059777325, 72.00060205939565],
    [-174.0008093445371, 72.00060192652217],
    [-174.0006053017384, -15.000601990025386],
    [-134.22597532324247, -15.000602004951233],
    [-106.00421062932261, -15.000602024344811],
    [-63.754083308670964, -15.00060201818635],
    [-26.290103199499303, -15.000601982388343],
    [34.939573443906426, -15.000602040237249],
    [91.87822164793576, -15.000602000123632],
    [132.47795335015223, -15.000602038567253],
    [164.

In [ ]:
# !pip install geemap

  Using cached earthengine_api-1.5.14-py3-none-any.whl.metadata (2.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_cloud_storage-3.1.0-py2.py3-none-any.whl.metadata (12 kB)
  Using cached google_api_python_client-2.169.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.40.1-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Usi

In [34]:
import geemap

In [77]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [78]:
map.addLayer(dem)

In [79]:
geometry = [ee.Geometry.Point([x,y],'EPSG:32119') for x,y in zip(data["X"], data["Y"])]

In [80]:
fc = ee.FeatureCollection(geometry)
map.addLayer(fc)

In [81]:
original_info = fc.getInfo()
original_info

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699102.8877924071, 186780.4458126684]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699102.8877924071, 186780.4458126684]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699105.8874190656, 186780.4458126684]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
    'type': 'Point',
    'coordinates': [699099.8881657487, 186777.44618600988]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'crs': 

In [82]:
sample_fc = dem.sampleRegions(
    collection=fc,
    scale=10, # Resolution of the image
    geometries=True
)

In [83]:
sample_fc

In [84]:
sample_info = sample_fc.getInfo()
sample_info

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['elevation']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-78.01426489169957, 35.429736096570515]},
   'id': '0_0',
   'properties': {'elevation': 22.24553871154785}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-78.01426489169957, 35.429736096570515]},
   'id': '1_0',
   'properties': {'elevation': 22.24553871154785}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-78.01417506017115, 35.429736096570515]},
   'id': '2_0',
   'properties': {'elevation': 22.477031707763672}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-78.01426489169957, 35.429736096570515]},
   'id': '3_0',
   'properties': {'elevation': 22.24553871154785}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    't

In [85]:
sample_info['features']

[{'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-78.01426489169957, 35.429736096570515]},
  'id': '0_0',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-78.01426489169957, 35.429736096570515]},
  'id': '1_0',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-78.01417506017115, 35.429736096570515]},
  'id': '2_0',
  'properties': {'elevation': 22.477031707763672}},
 {'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-78.01426489169957, 35.429736096570515]},
  'id': '3_0',
  'properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-78.01417506017115, 35.429736096570515]},
  'id': '4_0',
  'properties': {'elevation': 22.4770317

In [91]:
for ind,itm in enumerate(original_info['features']):
    itm['properties'] = sample_info['features'][ind]['properties']

In [92]:
original_info['features']

[{'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699102.8877924071, 186780.4458126684]},
  'id': '0',
  'properties': {'elevation': 22.24553871154785},
  '[properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699102.8877924071, 186780.4458126684]},
  'id': '1',
  'properties': {'elevation': 22.24553871154785},
  '[properties': {'elevation': 22.24553871154785}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',
   'coordinates': [699105.8874190656, 186780.4458126684]},
  'id': '2',
  'properties': {'elevation': 22.477031707763672},
  '[properties': {'elevation': 22.477031707763672}},
 {'type': 'Feature',
  'geometry': {'crs': {'type': 'name', 'properties': {'name': 'EPSG:32119'}},
   'type': 'Point',


## Create a feature class and add elevation data to the features

In [88]:
import os
fcname= os.path.join(arcpy.env.workspace, "pnt_elev.shp")
if arcpy.Exists(fcname):
    arcpy.management.Delete(fcname)
arcpy.management.CreateFeatureclass(arcpy.env.workspace, "pnt_elev.shp", "POINT", spatial_reference=sr)

<Result 'F:\\OneDrive - Louisiana State University\\LSU_Course\\GEOG\\GEOG 4057 GIS Programming\\GEOG4057_Hanqi\\Projects\\Project_2_Hanqi\\pnt_elev.shp'>

In [89]:
arcpy.management.AddField(fcname, "elevation", "FLOAT")

<Result 'F:\\OneDrive - Louisiana State University\\LSU_Course\\GEOG\\GEOG 4057 GIS Programming\\GEOG4057_Hanqi\\Projects\\Project_2_Hanqi\\pnt_elev.shp'>

In [93]:
with arcpy.da.InsertCursor(fcname, ["SHAPE@", "elevation"]) as cursor:
    for feature in original_info['features']:
        geom = feature['geometry']
        coords = geom['coordinates']
        x, y = coords[0], coords[1]
        point = arcpy.PointGeometry(arcpy.Point(x, y))
        prop = feature['properties']
        elev = prop['elevation']
        cursor.insertRow([point, elev])